In [1]:
import pandas as pd
f = open("car.txt.txt", encoding="utf8")
data = f.read()
data = data.split("\n")

In [2]:
data

['Unknown Speaker  1:27  ',
 'We are pushing forward.',
 '',
 'Unknown Speaker  1:32  ',
 "Checking and seems all side clear. Coming out wide, and I'm going to just take a waiting space, starting the signal, checking all side now",
 '',
 'Unknown Speaker  1:45  ',
 "Crossing, just seeing if pedestrians are walking towards which they are. So that just gives me the body language and they get to see the brake light, everyone's cued up. So don't need to speed up particularly",
 '',
 'Unknown Speaker  2:09  ',
 "So I'm going to be turning left here. So I'm already sort of keeping a bit of a space there is a cycle lane, so I can't be right up against it. And I'm also aware that I want a bit of space for cyclists to pass because it's nice.",
 '',
 'Unknown Speaker  2:27  ',
 "There's obviously a number of different styles of driving, if you like, I tend to do a mix of sort of advanced but also eco driving as well. So to marry together quite nicely. I was trying to say here, get in position on

In [3]:
data[34]

"A little bit of restriction with the bus or whether white vans decide not going round who's got plenty of room? G"

In [4]:
id = []
text = []
for i in range(len(data)):
  if (i % 3 == 0):
    id.append(data[i])
  elif (i % 3 == 1):
    text.append(data[i])

In [5]:
len(id),len(text)

(86, 86)

In [13]:
d = {'id': id, 'text': text}
df = pd.DataFrame(data=d)
df.drop(index = 85, inplace = True)

In [14]:
df

,id,text
0,Unknown Speaker 1:27,We are pushing forward.
1,Unknown Speaker 1:32,Checking and seems all side clear. Coming out ...
2,Unknown Speaker 1:45,"Crossing, just seeing if pedestrians are walki..."
3,Unknown Speaker 2:09,So I'm going to be turning left here. So I'm a...
4,Unknown Speaker 2:27,There's obviously a number of different styles...
...,...,...
80,Unknown Speaker 50:13,plenty of space between me and the car behind ...
81,Unknown Speaker 50:27,one of the primary schools just around here is...
82,Unknown Speaker 50:42,so how come you're give him a space with the c...
83,Unknown Speaker 50:49,"I'd love to be doing 30, but I'm trying to kee..."


In [15]:
# import spacy
# from spacy import displacy

# NER = spacy.load("en_core_web_sm")
# text1= NER(text[6])

# for i in text:
#   for word in NER(i).ents:
#     print(word.text,word.label_)

In [16]:
import re

def split_it(text):
    return re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text)
df['text'] = df['text'].apply(lambda x: split_it(x))
df['text'] = df['text'].apply(lambda x: x.replace('RT',''))

In [17]:
df

,id,text
0,Unknown Speaker 1:27,We are pushing forward
1,Unknown Speaker 1:32,Checking and seems all side clear Coming out ...
2,Unknown Speaker 1:45,Crossing just seeing if pedestrians are walki...
3,Unknown Speaker 2:09,So I m going to be turning left here So I m a...
4,Unknown Speaker 2:27,There s obviously a number of different styles...
...,...,...
80,Unknown Speaker 50:13,plenty of space between me and the car behind ...
81,Unknown Speaker 50:27,one of the primary schools just around here is...
82,Unknown Speaker 50:42,so how come you re give him a space with the c...
83,Unknown Speaker 50:49,I d love to be doing 30 but I m trying to kee...


In [21]:
import nltk
nltk.download('stopwords')

df['text'] = df['text'].str.replace('[^\w\s]','')
df['text'] = df['text'].apply(lambda x: x.lower())
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
df

,id,text
0,Unknown Speaker 1:27,pushing forward
1,Unknown Speaker 1:32,checking seems side clear coming wide going ta...
2,Unknown Speaker 1:45,crossing seeing pedestrians walking towards gi...
3,Unknown Speaker 2:09,going turning left already sort keeping bit sp...
4,Unknown Speaker 2:27,obviously number different styles driving like...
...,...,...
80,Unknown Speaker 50:13,plenty space car behind react
81,Unknown Speaker 50:27,one primary schools around tim berners lee went
82,Unknown Speaker 50:42,come give space car apart 20 mile hour zone 20
83,Unknown Speaker 50:49,love 30 trying keep legal see lights actually ...


In [42]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=5)
data_text = vectorizer.fit_transform(df['text'])
df1 = pd.DataFrame(data_text.toarray())

In [43]:
X = df1.copy()

In [47]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)

In [48]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3,
       1, 1, 1, 0, 2, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int32)

In [49]:
df['target'] = kmeans.labels_

In [50]:
df

,id,text,target
0,Unknown Speaker 1:27,pushing forward,1
1,Unknown Speaker 1:32,checking seems side clear coming wide going ta...,1
2,Unknown Speaker 1:45,crossing seeing pedestrians walking towards gi...,1
3,Unknown Speaker 2:09,going turning left already sort keeping bit sp...,1
4,Unknown Speaker 2:27,obviously number different styles driving like...,1
...,...,...,...
80,Unknown Speaker 50:13,plenty space car behind react,1
81,Unknown Speaker 50:27,one primary schools around tim berners lee went,1
82,Unknown Speaker 50:42,come give space car apart 20 mile hour zone 20,1
83,Unknown Speaker 50:49,love 30 trying keep legal see lights actually ...,1


In [51]:
df.to_csv('res.csv',index = False)